In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# An example of a Low Liquidity Trade

In [1]:
import requests
import s3fs
import pandas as pd

from datetime import timedelta, datetime
import pendulum

import yfinance as yf

from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker

# Homemade
from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData

from normalize.ticker_symbols import TsxToYhoo
from addFeatures.daily import DailyData 

from mocIO.read_moc import get_moc_file_lst
#from extractMOCData import read_moc 

In [2]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [3]:
moc_file_lst = get_moc_file_lst.run("tsx-moc")

In [4]:
data_flpth = f"s3://{moc_file_lst[-1]}"
data_flpth

's3://tsx-moc/moc_tsx_20200407.csv'

In [5]:
moc_df = pd.read_csv(data_flpth, parse_dates=["moc_date"], na_filter=False)
moc_df["Symbol"].fillna("NA", inplace=True)

In [6]:
moc_df[moc_df["Symbol"].isna()==True]

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date


## Add daily features
(from yahoo)

In [7]:
yhooMap =  TsxToYhoo()
moc_key_df = yhooMap.run(moc_df)
moc_key_df

PWF.PR.P
PWF.PR.T


,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date,yahoo_symbol
0,AAV,BUY,4356,1.575,2020-04-07,AAV.TO
1,AD,BUY,7220,8.670,2020-04-07,AD.TO
2,ADN,BUY,100,12.660,2020-04-07,ADN.TO
3,AFN,BUY,3046,17.325,2020-04-07,AFN.TO
4,AI,BUY,100,9.070,2020-04-07,AI.TO
...,...,...,...,...,...,...
399,WTE,SELL,2012,13.185,2020-04-07,WTE.TO
400,X,SELL,1400,107.640,2020-04-07,X.TO
401,YGR,SELL,1000,0.488,2020-04-07,YGR.TO
402,YRI,SELL,179254,4.695,2020-04-07,YRI.TO


In [51]:
# Create price data _list
symbol_clmn="Symbol"
dailyData =  DailyData(symbol_clmn_nm=symbol_clmn)
eod_df = dailyData.get_eod_data()

/home/ilivni/MOC/addFeatures/daily.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
INFO:root:Error getting info from yahoo for sym ABX.TO
INFO:root:Error getting info from yahoo for sym ABX.TO
INFO:root:Error getting info from yahoo for sym AC.TO
INFO:root:Error getting info from yahoo for sym AD.TO


In [52]:
moc_key_df

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date,yahoo_symbol
0,ABX,BUY,338560,29.265,2020-04-06,ABX.TO
1,AC,BUY,1231,15.080,2020-04-06,AC.TO
2,ACB,BUY,117978,1.145,2020-04-06,ACB.TO
3,AD,BUY,5313,7.685,2020-04-06,AD.TO
4,ADN,BUY,100,12.100,2020-04-06,ADN.TO


In [53]:
ohlc_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,yahoo_symbol,sector,currency,marketCap,sharesShort,floatShares,enterpriseValue,exchangeTimezoneName,forwardPE
0,2020-04-06,29.01,29.86,28.54,29.06,4571300,0,0,ABX.TO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-06,15.24,15.49,14.86,15.16,5717319,0,0,AC.TO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-06,1.20,1.22,1.11,1.12,9443200,0,0,ACB.TO,Healthcare,CAD,1.309246e+09,18853459.0,1.153813e+09,1.720738e+09,America/Toronto,-8.615385
3,2020-04-06,7.84,8.15,7.65,7.76,360100,0,0,AD.TO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-06,12.77,12.77,12.00,12.16,32400,0,0,ADN.TO,Basic Materials,CAD,2.029127e+08,48331.0,8.800513e+06,2.938527e+08,America/Toronto,9.574803


In [135]:
def get_eod_data(grp, date, interval = "1m"):
    
#     print(grp)
    
    symbol_lst = grp["yahoo_symbol"].values.tolist()
    print(symbol_lst)
    
    st_dt = date
    print(st_dt)
    end_dt = st_dt + timedelta(days=1)
    print(end_dt)
    df = yf.download(
        symbol_lst, 
        start=st_dt.strftime('%Y-%m-%d'), 
        end=end_dt.strftime('%Y-%m-%d'),
        interval=interval,
        group_by = 'ticker'
    )
    
    return df

# 1. download EOD yahoo data date
grpd_eod_dfs = moc_key_df.groupby(by=["moc_date"])
for grp in grpd_eod_dfs:
    df = get_eod_data(grp[1], grp[0], interval="1d")

['ABX.TO', 'AC.TO', 'ACB.TO', 'AD.TO', 'ADN.TO']
2020-04-06 00:00:00
2020-04-07 00:00:00
[*********************100%***********************]  5 of 5 completed


In [146]:
df.stack(level=0)

Adj Close      Close       High        Low   Open   Volume
Date                                                                         
2020-04-06 ABX.TO  29.059999  29.059999  29.860001  28.540001  29.01  4571300
           AC.TO   15.160000  15.160000  15.490000  14.860000  15.24  5717319
           ACB.TO   1.120000   1.120000   1.220000   1.110000   1.20  9443200
           AD.TO    7.760000   7.760000   8.150000   7.650000   7.84   360100
           ADN.TO  12.160000  12.160000  12.770000  12.000000  12.77    32400

In [132]:
 df = yf.download(
        
        ['ABX.TO' 'AC.TO' 'ACB.TO' 'AD.TO' 'ADN.TO'], 
        start=st_dt.strftime('%Y-%m-%d'), 
        end=end_dt.strftime('%Y-%m-%d'),
        interval=interval
 )

NameError: name 'st_dt' is not defined

In [111]:
for i in grpd_eod_dfs:
    print(i[1]["yahoo_symbol"])

0    ABX.TO
1     AC.TO
2    ACB.TO
3     AD.TO
4    ADN.TO
Name: yahoo_symbol, dtype: object


In [ ]:
data = yf.download(
    "SPY AAPL", 
    start="2020-04-06", 
    end="2017-04-30"
)

In [55]:
# Make daily moc data
daily_moc_df = moc_key_df.merge(
    ohlc_df,
    how="left",
    left_on=["moc_date", "yahoo_symbol"],
    right_on=["Date", "yahoo_symbol"],
    validate="one_to_one"
)

In [60]:
daily_moc_df = daily_moc_df.drop("Date", axis=1)
daily_moc_df

KeyError: "['Date'] not found in axis"

In [59]:
daily_moc_df.columns

Index(['Symbol', 'Imbalance Side', 'Imbalance Size',
       'Imbalance Reference Price', 'moc_date', 'yahoo_symbol', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'sector',
       'currency', 'marketCap', 'sharesShort', 'floatShares',
       'enterpriseValue', 'exchangeTimezoneName', 'forwardPE'],
      dtype='object')

In [71]:
clmn_name = 'Imbalance Side'
tmp_col = clmn_name.split(" ")
tmp_col[0].lower()

'imbalance'

In [ ]:
def norm_clm_names(clm_name):
    clm_name_lst = 

In [68]:
daily_moc_df.columns  = map(str.split(" ") ,daily_moc_df.columns) 

TypeError: 'list' object is not callable

In [67]:
som

In [11]:
ohlc_df_lst = dailyData.add_ohlc(moc_key_df)

- ITP.TO: Request Failed
- TRST.TO: No data found for this date range, symbol may be delisted


In [12]:
df_lst = [df for df in ohlc_df_lst]

In [13]:
ohlc_df = pd.concat(df_lst, axis=0).reset_index()

In [14]:
ohlc_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,yahoo_symbol,Adj Close
0,2020-04-06,29.01,29.86,28.54,29.06,4573154.0,0.0,0.0,ABX.TO,NaN
1,2020-04-06,15.24,15.49,14.86,15.16,5715770.0,0.0,0.0,AC.TO,NaN
2,2020-04-06,1.20,1.22,1.11,1.12,9443236.0,0.0,0.0,ACB.TO,NaN
3,2020-04-06,7.84,8.15,7.65,7.76,359895.0,0.0,0.0,AD.TO,NaN
4,2020-04-06,12.77,12.77,12.00,12.16,32392.0,0.0,0.0,ADN.TO,NaN


In [ ]:
st_dt = moc_key_df.iloc[0]["moc_date"].strftime('%Y-%m-%d')
st_dt = "2020-04-02"

In [22]:
sym = yf.Ticker("AEM.TO")

In [36]:
sym.info["sector"]

'Basic Materials'

In [72]:
sym.history(
    start="2020-04-02",
    end="2020-04-03",
    interval = "1m",
    auto_adjust = True
)
    

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2020-04-02 09:30:00-04:00,60.35,61.10,60.35,60.93,0,0,0
2020-04-02 09:31:00-04:00,60.96,60.96,60.10,60.10,1323,0,0
2020-04-02 09:32:00-04:00,60.73,60.75,60.70,60.70,982,0,0
2020-04-02 09:33:00-04:00,60.90,60.90,60.22,60.22,2324,0,0
2020-04-02 09:34:00-04:00,60.68,60.68,60.68,60.68,951,0,0
...,...,...,...,...,...,...,...
2020-04-02 15:55:00-04:00,62.32,62.33,62.09,62.23,12404,0,0
2020-04-02 15:56:00-04:00,62.24,62.51,62.18,62.27,16983,0,0
2020-04-02 15:57:00-04:00,62.27,62.27,61.83,62.08,26340,0,0


In [48]:
sym.ticker

'AEM.TO'

In [46]:
def get_ohlc(row):
    sym = yf.Ticker(row["yahoo_symbol"])
    print(sym.info)
    st_dt = row["moc_date"]
    end_dt = st_dt + timedelta(days=1)
    
    df = sym.history(
            start=st_dt.strftime('%Y-%m-%d'), 
            end=end_dt.strftime('%Y-%m-%d'), 
            auto_adjust=True
        ).head(1)
    
    # Add symbolto ohlc
    df["yahoo_symbol"] = row["yahoo_symbol"]
    print(sym.info)
    #df["sector"] = sym.info["sector"]
    return df

ohlc_df_lst = moc_key_df.head(2).apply(get_ohlc, axis=1)

IndexError: list index out of range

In [ ]:
ohlc_df_lst[0].rea

In [ ]:
st_dt = moc_key_df.iloc[0]["moc_date"].strftime('%Y-%m-%d')
end_dt = moc_key_df.iloc[0]["moc_date"] # + timedelta(days=1)
st_dt

In [ ]:
sym_to_get_lst[:2]

In [ ]:
data

In [ ]:
return sym.history(start=st_dt).head(1)

In [ ]:
def get_eod_data(row):
    sym = yf.Ticker(row["yahoo_tsx_symbol"])
    row["currency"] = sym.info["currency"]
    
appiled_df = df[""].apply(get_eod_data, axis=1)
df = pd.concat([moc_key_df, appiled_df], axis='columns')

In [ ]:
sym =  moc_key_df.iloc[0]["Symbol"]
sym

In [ ]:
pre_client = Client()
project_nm = "MOC"

In [ ]:
try:
    proj_id = pre_client.create_project(project_name=project_nm)
except Exception as err:
    print(err.with_traceback)

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")